<a href="https://colab.research.google.com/github/vinnimodi/Automatic-Question-Generator/blob/main/Final_variable_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U nltk
!pip install -U textblob
!python -m textblob.download_corpora

import random 
import string

import re
from textblob import TextBlob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 5.1 MB/s 
  Attempting uninstall: textblob
    Found existing installation: textblob 0.15.3
    Uninstalling textblob-0.15.3:
      Successfully uninstalled textblob-0.15.3
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_r

In [ ]:
data = '''
It's not great if its 1 st letter in variable name is digits.
Its good to have 2 nd letter as capital.
Variable can contain digits for good.
It would be better if it has atleast 1 capital letter.
Variable should contain maximum 8 alphabet for good variable name.
Variable should contain exactly 2 digits to pass.
variable should contain underscore to great.
Variable can contain alphabet for good.
variable should not contain special characters for worst.
'''
data = TextBlob(data)
lines = {}
for sentence in data.sentences:    
    keyword = {}
    lines[sentence.string] = keyword;
    for i in sentence.tags:
        key = i[1].encode('utf-8')
        if key not in keyword:
            keyword[key] = []
        keyword[key].append(i[0].encode('utf-8'))

def decode(l):
    if isinstance(l, list):
        return [decode(x) for x in l]
    else:
        return l.decode('utf-8')
def questions(word, sentence):
    change= re.compile(re.escape(word), re.IGNORECASE)
    return change.sub('________', sentence)
position = {}
minm={}
maxm={}
exact={}
special={}
dic={}
count=['atleast','maximum','minimum','exactly']
ch=['alphabet','capital','digits','special character','alphanumeric']
sp=["!", "”", "#", "$", "%", "&", "’", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "]", "^", "_", "`", "{", "|", "}", "~"]
spname=["Exclamation", "Double quote", "hash", "Dollar", "Percent", "Ampersand", "Single quote", "Left parenthesis", "Right parenthesis", "Asterisk", "Plus", "Comma", "Minus", "Full stop", "Slash", "Colon", "Semicolon", "Less than", "Equal sign", "Greater than", "Question mark", "At sign", "Left bracket", "Right bracket", "Caret", "underscore", "backtick", "Left brace", "Vertical bar", "Right brace", "Tilde"]
def frame(sentence,poss):
    analyse = TextBlob(sentence)
    val=analyse.sentiment[0]
    # if(val>0):
    #   print(sentence+': positive')
    # else:
    #   print(sentence+': negative')
    for i in range(len(spname)):
      if(sentence.find(spname[i])!=-1):
        special[spname[i]]=sp[i]
        if(val>0):
          dic[spname[i]]=True
        else:
          dic[spname[i]]=False


    if b'CD' in poss:
        words = decode(poss[b'CD']) #The digit e.g. 2
        c=0
        for i in count:
           if(sentence.find(i)!=-1):
              c=1
              for j in ch:
                if(sentence.find(j)!=-1):
                  if(i=='exactly'):
                    exact[words[0]]=j
                  elif(i=='minimum'or i=='atleast'):
                    minm[words[0]]=j
                  else:
                    maxm[words[0]]=j 
        if(c!=1):
          for i in ch:
              if(sentence.find(i)!=-1):
                c=1
                if(val>0):
                  position[words[0]]=str(i)+" True"
                else:
                  position[words[0]]=str(i)+" False"
    else:
       for i in ch:
         if(sentence.find(i)!=-1):
           if(val>0):
             dic[i]=True
           else:
             dic[i]=False

# Iterate over the sentences 
for sentence in lines.keys():
    keyword = lines[sentence]
    frame(sentence,keyword)
# alpha-numeric-special-check
def anspcheck(variable): 
  return variable.isalnum()
def ccheck(variable):
  return  any(c.isupper() for c in variable)
def acheck(variable):
  return any(c.isalpha() for c in variable)
def dcheck(variable):
  return any(c.isdigit() for c in variable)
def spcheck(variable):
  for i in sp:
    if(i in variable):
      return True
  return False


def specialcheck(variable):
  a=True
  for i in range(len(sp)):
    if (sp[i] in variable):
      if(spname[i] in dic):
        if(dic[spname[i]]==False):
          a=not(sp[i] in variable)
          if(a==False):
            return a
      else:
        return False
  return a

print(position)
print(minm)
print(maxm)
print(exact)
print(special)
print(dic)

def simplify(variable):
  count=0
  for i in ch:
    if i not in dic:
      dic[i]=True
      count=count+1
    else:
      if(dic[i]==True):
        count=count+1
  if(count==5):
    return True
  else:
    if(dic["alphanumeric"]==True and dic["special character"]==False):
      ans=anspcheck(variable)
      ans=specialcheck(variable)
      if(ans==True):
        if(dic["capital"]==True):
          return ccheck(variable)
        else:
          return not(ccheck(variable))
      else:
        return ans
    elif(dic["alphanumeric"]==True):
      if(dic["capital"]==True):
        return ccheck(variable)
      else:
        return not(ccheck(variable))
    elif(dic["alphanumeric"]==False and dic["special character"]==False):
      ans=True
      if(dic["alphabet"]==True):
        ans=acheck(variable)
        if(ans==False):
          return False
      else:
        ans=not(acheck(variable))
        if(ans==False):
          return False
      if(dic["digits"]==True):
        ans=dcheck(variable)
        if(ans==False):
          return False
      else:
        ans=not(dcheck(variable))
        if(ans==False):
          return False
      if(dic["capital"]==True):
        ans=ccheck(variable)
        if(ans==False):
          return False
      else:
        ans=not(ccheck(variable))
        if(ans==False):
          return False
      return ans

def poscheck(variable):
  if(bool(position)==False):
    return True
  ans=True
  for p,val in position.items():
    if("digits" in val):
      v=val[-5:]
      
      if(v==" True"):
        ans=dcheck(variable[int(p)-1])
        if(ans==False):
          return ans
      else:
        ans=not(dcheck(variable[int(p)-1]))
        if(ans==False):
          return ans
    if("alphabet" in val):
      v=val[-5:]
      
      if(v==" True"):
        ans=acheck(variable[int(p)-1])
        if(ans==False):
          return ans
      else:
        ans=not(acheck(variable[int(p)-1]))
        if(ans==False):
          return ans
    if("capital" in val):
      v=val[-5:]
      
      if(v==" True"):
        ans=ccheck(variable[int(p)-1])
        if(ans==False):
          return ans
      else:
        ans=not(ccheck(variable[int(p)-1]))
        if(ans==False):
          return ans
    if("special character" in val):
      v=val[-5:]
      
      if(v==" True"):
        ans=spcheck(variable[int(p)-1])
        if(ans==False):
          return ans
      else:
        ans=not(spcheck(variable[int(p)-1]))
        if(ans==False):
          return ans

  return ans

def freqcheck(variable):
  if(bool(exact)==False and bool(minm)==False and bool(maxm)==False):
    return True
  ans=True
  c1=0
  c2=0
  c3=0
  c4=0
  for i in variable:
    if i.isdigit():
      c1=c1+1
    elif i.isalpha():
      c2=c2+1
      if i.isupper():
        c3=c3+1
    else:
      c4=c4+1
  for p,val in exact.items():
    if(val=="digits"):
      if(c1!=int(p)):
        return False
    if(val=="alphabet"):
      if(c2!=int(p)):
        return False
    if(val=="capital"):
      if(c3!=int(p)):
        return False
    if(val=="special character"):
      if(c4!=int(p)):
        return False

  for p,val in minm.items(): 
    if(val=="digits"):
      if(c1<int(p)):
        return False
    if(val=="alphabet"):
      if(c2<int(p)):
        return False
    if(val=="capital"):
      if(c3<int(p)):
        return False
    if(val=="special character"):
      if(c4<int(p)):
        return False
  for p,val in maxm.items():
    if(val=="digits"):
      if(c1>int(p)):
        return False
    if(val=="alphabet"):
      if(c2>int(p)):
        return False
    if(val=="capital"):
      if(c3>int(p)):
        return False
    if(val=="special character"):
      if(c4>int(p)):
        return False
  return ans

def checker(variable):
  ans=simplify(variable)
  if(ans==True):
    if(dic["special character"]==False):
      ans=specialcheck(variable)
    if(ans==True):
      ans=poscheck(variable)
      if(ans==True):
        ans=freqcheck(variable)
  #     else:
  #       return ans
  #   else:
  #     return ans
  # else:
  #   return ans
  return ans


{'1': 'digits False', '2': 'capital True'}
{'1': 'capital'}
{'8': 'alphabet'}
{'2': 'digits'}
{'underscore': '_'}
{'digits': True, 'underscore': True, 'alphabet': True, 'special character': False}


In [ ]:
user_input = input("Enter variable name  to check validity: ")
print(checker(user_input))

Enter variable name  to check validity: aA12s
True


# VARIABLE QUESTION GENERATION

In [ ]:
def false_var_name_gen():
    addition = []
    addition += random.choices([*string.ascii_uppercase],k=random.randint(1,2))
    addition += random.choices([*string.punctuation],k=random.randint(1,2))
    addition += random.choices([*string.digits],k=random.randint(2,3))
    addition += random.choices([*string.ascii_lowercase],k=(10-len(addition)))
    random.shuffle(addition)
    return "".join(addition)

allchoice = []
dic["capital"] = True
for typech in dic:
  if(dic[typech]==True):
    allchoice.append(typech)

def getcharacter(type):
  if(type=="digits"):
    return random.choice([*string.digits])
  elif(type=="alphabet"):
    return random.choice([*string.ascii_lowercase])
  elif(type=="underscore"):
    return "_"
  elif(type=="capital"):
    return random.choice([*string.ascii_uppercase])
  elif(type=="special character"):
    return random.choice([*string.punctuation])

    

def true_var_name_gen():
    ans,rest=[None for i in range(10)],[]
    if(bool(position)==True):
      n_dig, n_low, n_cap, n_sp, n_u =0,0,0,0,0
      for p,val in position.items():
        if("digits" in val):
          v=val[-5:]
          if(v==" True"):
            ind = int(p)-1
            ans[ind]=random.choice(string.digits)
            n_dig+=1
          else:
            templist=allchoice.copy()
            templist.remove("digits")
            ind = int(p)-1
            ans[ind]=getcharacter(random.choice(templist))
            
        if("alphabet" in val):
          v=val[-5:]  
          if(v==" True"):
            ind = int(p)-1
            ans[ind]=random.choice(string.ascii_lowercase)
            n_low+=1
          else:
            templist=allchoice.copy()
            templist.remove("alphabet")
            ind = int(p)-1
            ans[ind]=getcharacter(random.choice(templist))
        if("underscore" in val):
          v=val[-5:]  
          if(v==" True"):
            ind = int(p)-1
            ans[ind]="_"
            n_low+=1
          else:
            templist=allchoice.copy()
            templist.remove("underscore")
            ind = int(p)-1
            ans[ind]=getcharacter(random.choice(templist))
        if("capital" in val):
          v=val[-5:]
          if(v==" True"):
            ind = int(p)-1
            ans[ind]=random.choice(string.ascii_uppercase)
            n_cap+=1
          else:
            templist=allchoice.copy()
            templist.remove("capital")
            ind = int(p)-1
            ans[ind]=getcharacter(random.choice(templist))
            
        if("special character" in val):
          v=val[-5:]
          if(v==" True"):
            ind = int(p)-1
            ans[ind]=random.choice(string.punctuation)
            n_sp+=1
          else:
            templist=allchoice.copy()
            templist.remove("special character")
            ind = int(p)-1
            ans[ind]=getcharacter(random.choice(templist))
      if(not(bool(exact)==False and bool(minm)==False and bool(maxm)==False)):
        for p,val in exact.items():
          if(val=="digits"):
            rest+=[random.choice(string.digits) for i in range(int(p)-n_dig)]
            n_dig+=int(p)-n_dig
          if(val=="alphabet"):
            rest+=[random.choice(string.ascii_lowercase) for i in range(int(p)-n_low)]
            n_low+=int(p)-n_low
          if(val=="capital"):
            rest+=[random.choice(string.ascii_uppercase) for i in range(int(p)-n_cap)]
            n_cap+=int(p)-n_cap
          if(val=="special character"):
            rest+=[random.choice(string.punctuation) for i in range(int(p)-n_sp)]
            n_sp+=int(p)-n_sp

        for p,val in minm.items():
          if(val=="digits"):
            if(n_dig<int(p)):
              rest+=[random.choice(string.digits) for i in range(int(p)-n_dig)]
          if(val=="alphabet"):
            if(n_low<int(p)):
              rest+=[random.choice(string.ascii_lowercase) for i in range(int(p)-n_low)]
          if(val=="capital"):
            if(n_cap<int(p)):
              rest+=[random.choice(string.ascii_uppercase) for i in range(int(p)-n_cap)]
          if(val=="special character"):
            if(n_sp<int(p)):
              rest+=[random.choice(string.punctuation) for i in range(int(p)-n_sp)]
    for index, ch in enumerate(ans):
      if (ch==None):
        newch = rest.pop() if len(rest) > 0 else random.choice(string.ascii_lowercase)
        ans[index]= newch
    return "".join(ans)

In [ ]:
def final_var_gen(n):
    return [random.choice([false_var_name_gen(),true_var_name_gen()]) for i in range(n)]
n = 50 #no of questions
arr = final_var_gen(n)
arr = [i for i in filter(lambda a:isinstance(a, str),arr)]
## Testing
print(*arr, sep = "\n")

print("The inputed variable validity is:")
true_count,false_count = 0,0
for i in arr:
  if checker(i)==True:
    true_count+=1 
  else:
    false_count+=1
print("True: "+ str(true_count)+"/"+str(true_count+ false_count))
print("False: "+ str(false_count)+"/"+str(true_count+ false_count))

JF50zomcxq
3rfpM8kq~c
IL22pjruet
_P81opqqoq
pX52cejbve
_Y47uffuei
DP07vtvtek
IF50fkdhtp
FF11urdnas
ZA90cwnrdj
cP81lejlyb
,qOP0q2tnm
n9SiKz@8r7
63zJynt(Xs
XS4r7$+kep
qqcxy3!mX6
)fc8SV5|zd
mV03lqmdgt
rUX_k5v01j
LC34gielgw
d"ct1He2dx
)4k0vqLrl=
XI19eyvibj
Z13ddqj;vg
_W51sktdrz
_P44vnjdea
PJ22pmuecz
5U0nsmugtg
3S9dsuxvsl
BQ95qnqojv
3D5hlyhuif
OJ16pfhqio
1Q9uyyqhbr
YD06ohjjcu
tJt3a0^vhz
iL57aayvwp
_W93azuvck
oZ56syqkui
ue309&yL)d
6M9ebcwwcz
_D28vyepox
MC94hanctc
ZG98jcamoa
TP64tzmuur
vIt"vta63d
9G1bcdhoty
MN90spwvde
8N0mfgqpqu
GL62vrknep
_I54keilhr
The inputed variable validity is:
True: 30/50
False: 20/50


In [ ]:
print(data)
print("So check if the variable is valid or not.\n")
print("Type Yes valid and No for invalid.\n")
count = 1
for i in arr:
  abool = None
  ans = input((str(count) + ") " + i +" is valid?  "))
  # print(checker(i))
  if (ans =="Yes" or ans =="yes"):
    abool = True
  elif (ans=="no" or ans =="No"):
    abool=False
  else:
    print("Invalid answer")
    continue
  if abool ==checker(i):
    print("    CORRECT ANSWER\n")
  elif abool !=checker(i):
    print("   SORRY. WRONG ANSWER\n")
  count += 1



It's not great if its 1 st letter in variable name is digits.
Its good to have 2 nd letter as capital.
Variable can contain digits for good.
It would be better if it has atleast 1 capital letter.
Variable should contain maximum 8 alphabet for good variable name.
Variable should contain exactly 2 digits to pass.
variable should contain underscore to great.
Variable can contain alphabet for good.
variable should not contain special characters for worst.

So check if the variable is valid or not.

Type Yes valid and No for invalid.



KeyboardInterrupt: ignored